## PACK INSTALLATION (Restart kernel after run)

In [ ]:
# Install the scikit-learn library for machine learning algorithms
!pip install scikit-learn

In [ ]:
# Install the xlsxwriter library for creating Excel files
!pip install xlsxwriter

## IMPORT

In [ ]:
# Import OpenCV and check its version
import cv2
cv2.__version__

In [ ]:
# Import necessary libraries for image processing, visualization, and machine learning
from PIL import Image

# Napari for interactive visualization
import napari

# General-purpose libraries
import scipy
import csv
import imghdr
import colorsys
import os
import xlsxwriter

# Numerical computations
import numpy as np

# OpenCV for image processing
import cv2 as cv
import matplotlib.pyplot as plt

# PyVista for 3D visualization
import pyvista as pv

# Stardist for deep learning-based segmentation
from csbdeep.utils import normalize
from stardist.models import StarDist2D, Config3D, StarDist3D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label

# Scipy and Skimage for image processing and segmentation
from scipy import ndimage as ndi

from skimage import filters
from skimage.draw import disk
from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt
from skimage.measure import label, regionprops
from skimage.filters import threshold_local
from scipy.ndimage import center_of_mass

# from skimage.feature import peak_local_max
from vispy.color import Colormap
from IPython.display import display_html
from IPython.display import clear_output

from stardist.models import StarDist2D, Config3D, StarDist3D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label

import sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
from sklearn.svm import OneClassSVM

# Morphological operations for image processing
from skimage.morphology import remove_small_holes, remove_small_objects
from skimage.morphology import closing, square

# Napari settings for interactive visualization
from napari.settings import get_settings
settings = get_settings()

# AICSImageIO for reading and writing image files
from aicsimageio import AICSImage
from tifffile import imwrite

## Functions

In [ ]:
def ellipse_pixels(imarray, center, semi_axes, rotation, image_shape):
    """
    Generate the pixel coordinates that lie inside an ellipse.

    Parameters:
    - center: Tuple (x, y) representing the center of the ellipse.
    - semi_axes: Tuple (semi_major_axis, semi_minor_axis) representing the lengths of the ellipse's axes.
    - rotation: Rotation angle of the ellipse in radians.
    - image_shape: Shape of the image (height, width) to constrain the ellipse.

    Returns:
    - A 2D array of pixel coordinates (row, column) that lie inside the ellipse.
    """
    # Create a grid of x and y coordinates for the image
    y, x = np.meshgrid(np.arange(imarray.shape[1]), np.arange(imarray.shape[0]), indexing='xy')
    
    # Compute the cosine and sine of the rotation angle
    cos_theta = np.cos(rotation)
    sin_theta = np.sin(rotation)
    
    # Rotate the x and y coordinates to align with the ellipse's axes
    x_rot = cos_theta * (x - center[0]) + sin_theta * (y - center[1])
    y_rot = -sin_theta * (x - center[0]) + cos_theta * (y - center[1])

    # Create a mask for pixels that lie inside the ellipse
    mask = (x_rot / semi_axes[0])**2 + (y_rot / semi_axes[1])**2 <= 1

    # Enable interactive mode for Napari (if needed)
    settings.application.ipy_interactive = True

    # Return the coordinates of the pixels inside the ellipse
    return np.column_stack(np.where(mask))

def rgb_to_hsv(rgb_array):
    """
    Convert an array of RGB values to HSV (Hue, Saturation, Value).

    Parameters:
    - rgb_array: A 2D array where each row is an RGB triplet (R, G, B).

    Returns:
    - A 2D array where each row is an HSV triplet (H, S, V).
      H is in degrees (0-360), S and V are percentages (0-100).
    """
    hsv_list = []
    for rgb in rgb_array:
        r, g, b = rgb  # Extract the RGB components
        h, s, v = colorsys.rgb_to_hsv(r, g, b)  # Convert RGB to HSV
        hsv_list.append([h * 360, s * 100, v * 100])  # Scale H to degrees, S and V to percentages
    return np.array(hsv_list)

def csv_to_matrix(file_path):
    """
    Read a CSV file and convert its contents into a matrix.

    Parameters:
    - file_path: Path to the CSV file.

    Returns:
    - A 2D list where each row corresponds to a row in the CSV file.
    """
    matrix = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Convert numeric values to integers, leave others as strings
            matrix.append([int(value) if value.replace('.', '', 1).isdigit() else value for value in row])
    return matrix

def classify_points(cloud_points, test_points, bandwidth=1.0):
    """
    Classify test points based on their similarity to a cloud of points using Kernel Density Estimation (KDE).

    Parameters:
    - cloud_points: A 2D array of points representing the training data.
    - test_points: A 2D array of points to classify.
    - bandwidth: Bandwidth parameter for the KDE (controls smoothness).

    Returns:
    - A 1D array of probabilities for each test point.
    """
    # Fit a Kernel Density Estimator to the cloud points
    kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
    kde.fit(cloud_points)
    
    # Evaluate the probability density for each test point
    log_density = kde.score_samples(test_points)
    
    # Convert log-density to probabilities
    probabilities = np.exp(log_density)
    return probabilities

def calibrate_image_resolution(image: np.ndarray):
    """
    Opens a Napari viewer for interactive calibration.
    The user draws a line over the scalebar using the Shapes layer (Line)
    and presses 'q' to confirm. After the viewer closes the function will
    ask for the real-world scalebar length (in µm) and return the pixel
    resolution in µm/pixel.

    This function will loop and offer a retry if no line was confirmed
    (to avoid the need to re-run the calling cell).
    """

    while True:
        viewer = napari.Viewer()
        viewer.add_image(image, name='image')
        shapes_layer = viewer.add_shapes(name='scalebar', shape_type='line')

        # Provide clear on-screen instructions before opening Napari
        settings.application.ipy_interactive = False
        print("\n🟢 Napari opened: draw a line over the scalebar using the Shapes tool (Line).")
        print("When finished press 'q' while the Napari window is focused to confirm the measurement.")

        result = {}

        @viewer.bind_key('q')
        def _on_confirm(event=None):
            """Callback triggered when 'q' is pressed."""
            if len(shapes_layer.data) == 0:
                print("⚠️ No line found! Please draw one first and press 'q' again.")
                return
            line_coords = shapes_layer.data[0]
            p1, p2 = line_coords
            pixel_length = np.linalg.norm(p2 - p1)
            print(f"Measured scalebar length: {pixel_length:.2f} pixels")
            result['pixel_length'] = pixel_length
            viewer.close()

        napari.run()  # Waits until viewer is closed

        # If user confirmed a line, proceed to ask for real-world length
        if 'pixel_length' in result:
            try:
                real_length_um = float(input("Enter the real-world scalebar length (in µm): "))
            except Exception:
                raise ValueError("Invalid real-world length entered.")
            resolution_um_per_px = real_length_um / result['pixel_length']
            print(f"\n✅ Pixel resolution: {resolution_um_per_px:.4f} µm/pixel")
            return resolution_um_per_px

        # If no line was confirmed, offer a retry to the user
        print("No scalebar line was confirmed in Napari.")
        retry = input("Retry calibration? (y/n): ").strip().lower()
        if retry != 'y':
            raise ValueError("Calibration aborted by user. No line was confirmed.")

## Training image
Choose the image that will be used as training. It will be opened in Napari. Choose the circular selection on the right and choose as many nuclei as possible to train the algorithm. Then close the Napari window to continue

In [ ]:
# Define the training file and extract its name without the extension
training_file = 'pAC026 D28 uM SafO slide7.jpg'
training_stem = os.path.splitext(os.path.basename(training_file))[0]

# Open the TIFF file as an image
img_train = Image.open(training_file)

# Convert the image to a NumPy array for further processing
imarray_train = np.array(img_train)

# Remove alpha channel if present
if imarray_train.shape[2] == 4:
    imarray_train = imarray_train[:, :, :3]

# Create a copy of the original image to define the ROI (Region of Interest)
ROI_train = imarray_train.copy()

# Initialize a mask image with the same dimensions as the green channel of the original image
mask_train = np.zeros(np.shape(imarray_train[:, :, :]))

In [ ]:
# Define parameters for the investigation
step = 30  # Size of the square for investigation
delta = 1  # Sensitivity for blue dominance
holes_threshold = 1000  # Minimum area to consider a hole
island_threshold = 1000  # Minimum size to consider an object

# Flag to determine whether to use an existing CSV file for training points
use_csv = True

In [ ]:
## Collect average brightness and contrast from ROI of training image

# Iterate through the image in steps to identify blue-dominant regions
for i in range(1 + step, imarray_train.shape[0], step):
    for j in range(1 + step, imarray_train.shape[1], step):
        # Check if the blue channel is dominant over both red and green channels
        if (
            np.mean(ROI_train[i-step:i+step, j-step:j+step, 0]) >= delta + np.mean(ROI_train[i-step:i+step, j-step:j+step, 1]) and
            np.mean(ROI_train[i-step:i+step, j-step:j+step, 0]) >= delta + np.mean(ROI_train[i-step:i+step, j-step:j+step, 2])
        ):
            # Mark the region as part of the mask
            mask_train[i:i+step, j:j+step, :] = 1
        
bright_train=np.mean(imarray_train[:,:,:])
bright_train_ROI=np.mean(imarray_train[mask_train==1])

bright_train_R=np.mean(imarray_train[:,:,0])
bright_train_G=np.mean(imarray_train[:,:,1])
bright_train_B=np.mean(imarray_train[:,:,2])

In [ ]:
## Apply constant brightness to the training image

# Only run if not using CSV or CSV does not exist
if not(use_csv and os.path.exists(training_stem + '_yes_points.csv')):
    for i in range(1, imarray_train.shape[0]-1, 1):
        for j in range(1, imarray_train.shape[1]-1, 1):
            step_x=5 
            step_y=5
            if mask_train[i,j,0]==1:
                if i<step_x:
                    step_x=i
                elif i>(imarray_train.shape[0]-step_x):
                    step_x=imarray_train.shape[0]-i
                if j<step_y:
                    step_y=j
                elif j>(imarray_train.shape[1]-step_y):
                    step_y=imarray_train.shape[1]-j
    
                bright_train_region=np.mean(imarray_train[i-step_x:i+step_x, j-step_y:j+step_y,:])
    
                for h in [0,1,2]:
                    if (ROI_train[i,j,h]-int(bright_train_region-bright_train))<256:
                        ROI_train[i,j,h]=ROI_train[i,j,h]-int(bright_train_region-bright_train)
                    else:
                        ROI_train[i,j,h]=255                      

In [ ]:
settings.application.ipy_interactive = True

# Only show Napari if not using CSV
if use_csv==False:
    viewer_0 = napari.Viewer()
    
    viewer_0.add_image(imarray_train, name='Original', 
                    colormap='grey', blending='additive')
    viewer_0.add_image(mask_train, name='Violet', 
                    colormap='green', blending='additive')
    viewer_0.add_image(ROI_train, name='ROI', 
                    colormap='grey', blending='additive')

In [ ]:
# Check if the CSV file with training points exists
if use_csv and os.path.exists(training_stem + '_yes_points.csv'):
    # Load training points from the CSV file
    yes_points = csv_to_matrix(training_stem + '_yes_points.csv')
else:
    # Initialize an empty list for training points
    yes_points = []
    
    # Create an empty label array for the ROI
    ROI_labels = np.zeros(imarray_train.shape)
    
    # Disable interactive mode for Napari
    settings.application.ipy_interactive = False
    
    # Open the training image in Napari for manual annotation
    viewer_c = napari.Viewer()
    viewer_c.add_image(imarray_train, name='Original', 
                       colormap='grey', blending='additive')
    
    # Callback function to handle shape addition in Napari
    def on_shape_added(layer, event):
        shapes_layer = event.source

    # Add a shapes layer for annotating nuclei
    shapes_layer = viewer_c.add_shapes(shape_type="circle", name="Nuclei training")
    
    # Connect the mouse drag event to the callback function
    viewer_c.mouse_drag_callbacks.append(on_shape_added)
    
    # Run Napari for manual annotation
    napari.run()

    # Process each annotated shape to extract training points
    for s in range(0, np.shape(shapes_layer.data)[0]):
        # Get the bounding box of the shape
        xmax = np.max(shapes_layer.data[s][:, 0])
        ymax = np.max(shapes_layer.data[s][:, 1])
        xmin = np.min(shapes_layer.data[s][:, 0])
        ymin = np.min(shapes_layer.data[s][:, 1])
    
        # Calculate ellipse parameters
        center = ((xmax + xmin) / 2, (ymax + ymin) / 2)  # Center of the ellipse
        semi_axes = (
            abs(xmax - xmin) / 2,  # Semi-major axis (height / 2)
            abs(ymax - ymin) / 2,  # Semi-minor axis (width / 2)
        )
        rotation = 0  # Napari's ellipses are axis-aligned by default
    
        # Get the shape of the training image
        image_shape = imarray_train.shape
    
        # Get the pixel coordinates inside the ellipse
        enclosed_pixels = ellipse_pixels(imarray_train, center, semi_axes, rotation, image_shape[:2])

        # If the CSV file exists, load the existing training points
        if os.path.exists(training_stem + '_yes_points.csv'):
            yes_points = csv_to_matrix(training_stem + '_yes_points.csv')
    
        # Add the RGB values of the enclosed pixels to the training points
        for i in range(enclosed_pixels.shape[0]):
            x = enclosed_pixels[i][0]
            y = enclosed_pixels[i][1]
            yes_points.append([ROI_train[x, y, 0], ROI_train[x, y, 1], ROI_train[x, y, 2]])

        # Save the updated training points to the CSV file
        with open(training_stem + '_yes_points.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(yes_points)

# Convert the training points to a NumPy array for further processing
yes_points = np.array(yes_points)

In [ ]:
## Visualize distribution of training points

# Plot RGB scatter plots for training points using plt.figure + add_subplot
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1,3,1)
ax1.scatter(yes_points[:,0], yes_points[:,1], c = yes_points/255)
ax1.set_xlim((0,256))
ax1.set_xlabel("RED")
ax1.set_ylim((0,256))
ax1.set_ylabel("GREEN")

ax2 = fig.add_subplot(1,3,2)
ax2.scatter(yes_points[:,0], yes_points[:,2], c = yes_points/255)
ax2.set_xlim((0,256))
ax2.set_xlabel("RED")
ax2.set_ylim((0,256))
ax2.set_ylabel("BLUE")

ax3 = fig.add_subplot(1,3,3)
ax3.scatter(yes_points[:,1], yes_points[:,2], c = yes_points/255)
ax3.set_xlim((0,256))
ax3.set_xlabel("GREEN")
ax3.set_ylim((0,256))
ax3.set_ylabel("BLUE")

plt.tight_layout()

In [ ]:
# Polar visualization of Hue vs Saturation/Value
yes_points_hsv = rgb_to_hsv(yes_points)
theta = np.deg2rad(yes_points_hsv[:, 0])  # Hue in radians
r_s = yes_points_hsv[:, 1] / 100.0
r_v = yes_points_hsv[:, 2] / 100.0

fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(1,2,1, projection='polar')
sc1 = ax1.scatter(theta, r_s, c=yes_points/255.0, s=18, alpha=0.9)
ax1.set_title('Hue vs Saturation (polar)')
ax1.set_ylim(0,1)
ax1.set_yticks([0.25,0.5,0.75,1.0])

ax2 = fig.add_subplot(1,2,2, projection='polar')
sc2 = ax2.scatter(theta, r_v, c=yes_points/255.0, s=18, alpha=0.9)
ax2.set_title('Hue vs Value (polar)')
ax2.set_ylim(0,1)
ax2.set_yticks([0,50,100,150,200,255])

plt.tight_layout()
plt.show()

## File upload & Calibration
1. Set `tiff_file` below to the image you want to analyze.
2. Run the cell that loads the image (it prints the image shape).
3. The next cell will open Napari for *calibration* — draw a line over the image scalebar using the Shapes tool (choose the `Line` shape).
   - After drawing the line, make sure the Napari window is focused and press `q` to confirm the measurement.
   - If you accidentally close Napari or forget to press `q`, the function will ask whether you want to retry (choose `y` to reopen Napari).
4. After confirming the line you'll be prompted to enter the real-world scalebar length in micrometers (µm).
5. Calibration returns `r_X` and `r_Y` (µm/pixel) used for downstream size calculations.

In [ ]:
# Define the TIFF file to be processed
tiff_file = 'pAC024 D28 uM SafO slide7.jpg'

# Extract the file name without the extension
tiff_stem = os.path.splitext(os.path.basename(tiff_file))[0]

# Open the TIFF file as an image
img = Image.open(tiff_file)

# Convert the image to a NumPy array for further processing
imarray = np.array(img)

# Print the shape of the image array (dimensions of the image)
print(imarray.shape)

In [ ]:
# Calibrate image resolution (opens Napari to draw scalebar line)
res_px = calibrate_image_resolution(imarray)
r_X = res_px
r_Y = res_px

## ROI (Region of Interest)
This section detects tissue areas by looking for blue-dominant regions (the stained tissue).
- Make sure calibration has completed and `r_X`, `r_Y` are set before running this section.
- The algorithm scans the image in blocks defined by `step` and flags blocks where the blue channel dominates.
- After mask creation the code will remove small holes/objects and apply brightness adjustment based on the training image.
- Visual checks: Napari will open and overlay the detected ROI in green; if the ROI looks wrong, tweak `step`, `delta`, `holes_threshold`, or `island_threshold`.

Variables: [step] will dictate the size of the square of investigation (smaller values will get a better resolution but slower run), [delta] will dictate the sensitivity of the blue level (smaller values lead to more false positive but higher values lead to more false negative).  

In [ ]:
# Remove alpha channel if present
if imarray.shape[2] == 4:
    imarray = imarray[:, :, :3]

# Create a copy of the original image to define the ROI (Region of Interest)
ROI_image = imarray.copy()

# Initialize a mask image with the same dimensions as the green channel of the original image
mask_image = np.zeros(np.shape(imarray[:, :, 1]))

# Iterate through the image in steps to identify blue-dominant regions
for i in range(1 + step, imarray.shape[0], step):
    for j in range(1 + step, imarray.shape[1], step):
        # Check if the blue channel is dominant over both red and green channels
        if (
            np.mean(imarray[i-step:i+step, j-step:j+step, 0]) >= delta + np.mean(imarray[i-step:i+step, j-step:j+step, 1]) and
            np.mean(imarray[i-step:i+step, j-step:j+step, 0]) >= delta + np.mean(imarray[i-step:i+step, j-step:j+step, 2])
        ):
            # Mark the region as part of the mask
            mask_image[i:i+step, j:j+step] = 1

# Adjust brightness in ROI based on training image brightness
for i in range(1, imarray.shape[0]-1, 1):
    for j in range(1, imarray.shape[1]-1, 1):
        step_x=5 
        step_y=5
        if mask_image[i,j]==1:
            if i<step_x:
                step_x=i
            elif i>(imarray.shape[0]-step_x):
                step_x=imarray.shape[0]-i
            if j<step_y:
                step_y=j
            elif j>(imarray.shape[1]-step_y):
                step_y=imarray.shape[1]-j

            bright_image=np.mean(imarray[i-step_x:i+step_x, j-step_y:j+step_y,:])

            for h in [0,1,2]:
                if (ROI_image[i,j,h]-int(bright_image-bright_train))<256:
                    ROI_image[i,j,h]=ROI_image[i,j,h]-int(bright_image-bright_train)
                else:
                    ROI_image[i,j,h]=255


# Fill small holes in the mask to create a more continuous region
mask_filled = remove_small_holes(mask_image.astype(int), area_threshold=holes_threshold, connectivity=1)

# Remove small isolated objects from the mask
mask_filled = remove_small_objects(mask_filled, min_size=island_threshold, connectivity=1)

# Apply the mask to the original image to isolate the ROI
ROI_image = ROI_image * np.stack([mask_filled] * 3, axis=2)

In [ ]:
settings.application.ipy_interactive = True

# Visualize original, mask, and ROI in Napari
viewer_0 = napari.Viewer()

viewer_0.add_image(imarray, name='Original', 
                colormap='grey', blending='additive')
viewer_0.add_image(mask_filled, name='Violet', 
                colormap='green', blending='additive')
viewer_0.add_image(ROI_image, name='ROI', 
                colormap='grey', blending='additive')

After you run the next section, a new Napari window will appear with highlighted in green all the nuclei in the ROI.

In [ ]:
# Enable interactive mode for Napari
settings.application.ipy_interactive = True

# Create a copy of the ROI image for processing
imarray0 = ROI_image.copy()

# Extract the red, green, and blue channels from the ROI image
imR = imarray0[:, :, 0]
imG = imarray0[:, :, 1]
imB = imarray0[:, :, 2]

# Convert the training points from RGB to HSV
yes_points_hsv = rgb_to_hsv(yes_points)

# Scale the HSV training points for better performance in the SVM
scaler = StandardScaler()
yes_points_scaled = scaler.fit_transform(yes_points_hsv)

# Set up the One-Class SVM for anomaly detection
sensitivity = 0.8  # Sensitivity level (e.g., 10% of points allowed as outliers)
clf = OneClassSVM(kernel='rbf', nu=sensitivity, gamma='scale')
clf.fit(yes_points_scaled)

# Initialize an empty array for the violet channel
imV = np.zeros(np.shape(imR))

# Initialize a counter for progress tracking
tval = 0

# Iterate through the image to classify each pixel
for i in range(step, np.shape(imV)[0]):
    for j in range(step, np.shape(imV)[1]):
        tval += 1
        # Process only pixels within the mask
        if mask_filled[i, j] > 0:
            # Extract RGB values of the current pixel
            pR = imR[i, j]
            pG = imG[i, j]
            pB = imB[i, j]
            X = np.array([[pR, pG, pB]])

            # Convert the pixel to HSV and scale it
            X_hsv = rgb_to_hsv(X)
            X_scaled = scaler.transform(X_hsv)

            # Classify the pixel using the KDE-based classifier
            predictions = classify_points(yes_points, X, bandwidth=20.0)

            # Assign the prediction value to the violet channel
            if predictions > 0.0:
                imV[i, j] = predictions[0]
            else:
                imV[i, j] = 0
        else:
            j += step  # Skip to the next step if outside the mask

        # Display progress at regular intervals
        if (100.0 * tval / (np.shape(imV)[0] * np.shape(imV)[1]) % 1.0 == 0.0):
            clear_output(wait=True)
            print('PROGRESS ' + str(100.0 * tval / (np.shape(imV)[0] * np.shape(imV)[1])) + ' %')

# Visualize the results in Napari
viewer_0 = napari.Viewer()

# Add the image to the viewer
viewer_0.add_image(imarray, name='Original', 
                colormap='grey', blending='additive')

# Add the violet channel (classification results) to the viewer
viewer_0.add_image(imV, name='Violet', 
                   colormap='green', blending='additive')


# Image processing

### Visualization

In [ ]:
viewer_full = napari.Viewer()
viewer_full.add_image(imarray, name='Original', colormap='grey', blending='additive')
viewer_full.add_image(imV, name='Violet', colormap='green', blending='additive')

### Step 1: Thresholding 
Switch from grayscale to binary scale (different brightness to either 0 for background or 1 for nuclei)

In [ ]:
im_in=imV

# Get all nonzero values in imV (ignoring background)
valid_pixels = imV[imV > 0]  

# Compute the percentile threshold only for these valid pixels
if len(valid_pixels) > 0:  # Ensure there are nonzero pixels
    threshold_value = np.percentile(valid_pixels, 95.0)
    strong_th_value = np.percentile(valid_pixels, 98.0)
    binary_mask = (imV >= threshold_value)  # Keep only the top 5% brightest pixels
    strong_b_mask = (imV >= strong_th_value)  # Keep only the top 5% brightest pixels
else:
    binary_mask = np.zeros_like(imV)  # If no valid pixels, return empty mask
    
viewer_full.add_image(binary_mask.astype(float), name="Thresholded", colormap="blue", blending="additive")
napari.run()
im_out=binary_mask.copy()

### Step 2: Remove Noise & Artifacts

In [ ]:
im_in=im_out
binary_mask=im_in

# Remove small objects and close gaps
binary_mask2 = remove_small_objects(binary_mask, min_size=5)  
binary_mask2 = closing(binary_mask2, square(3))  # Fills small holes and connects nearby white regions.

strong_b_mask2 = remove_small_objects(strong_b_mask, min_size=5)  
strong_b_mask2 = closing(strong_b_mask2, square(3))  # Fills small holes and connects nearby white regions.

viewer_full.add_image(binary_mask2.astype(float), name="Noise and art. removed", colormap="blue", blending="additive")
napari.run()
im_out=binary_mask2.copy()

### Step 3: Remove Artifacts Based on Size and Roundness

In [ ]:
im_in=im_out
binary_mask2=im_in

min_roundness = 0.75

# Define the minimum and maximum size for nuclei to be considered valid
min_nucleus_diam = 3  # Minimum diameter of a nucleus (um)
max_nucleus_diam = 20  # Maximum diameter of a nucleus (um)

min_nucleus_diam_res = min_nucleus_diam/((r_X+r_Y)/2)
max_nucleus_diam_res = max_nucleus_diam/((r_X+r_Y)/2)

min_nucleus_size=(min_nucleus_diam/2)*(min_nucleus_diam/2)*np.pi
max_nucleus_size=(max_nucleus_diam/2)*(max_nucleus_diam/2)*np.pi

# Initialize an empty mask to store the filtered regions
filtered_mask = np.zeros_like(binary_mask2)
large_nuclei_mask = filtered_mask.copy()

# Iterate through each connected region in the labeled binary mask
for region in regionprops(label(binary_mask2)):
    # Check if the region's area falls within the valid size range
    if min_nucleus_size <= region.area <= max_nucleus_size:
        if region.perimeter > 0:
            # Compute roundness (circularity)
            roundness = 4 * np.pi * region.area / (region.perimeter ** 2)

            # Keep only regions that are sufficiently round
            if roundness >= min_roundness:
                # Add the region to the filtered mask
                filtered_mask[label(binary_mask2) == region.label] = 1

# Iterate through each connected region in the labeled binary mask
for region in regionprops(label(strong_b_mask2)):
    # Check if the region's area falls within the valid size range
    if min_nucleus_size <= region.area <= max_nucleus_size:
        if region.perimeter > 0:
            # Compute roundness (circularity)
            roundness = 4 * np.pi * region.area / (region.perimeter ** 2)

            # Keep only regions that are sufficiently round
            if roundness >= min_roundness:
                # Add the region to the filtered mask
                filtered_mask[label(strong_b_mask2) == region.label] = 1

viewer_full.add_image(filtered_mask.astype(float), name="Size and roundness threshold", colormap="blue", blending="additive")
napari.run()
im_out=filtered_mask.copy()

In [ ]:
# #🔹 Step 4: Filter by Size and Roundness
# im_in = im_out
# segmented_nuclei = im_in

# # Define the minimum roundness value (1.0 = perfect circle)
# min_roundness = 0.75  # adjust as needed (0.7–0.85 works well for nearly round nuclei)

# # Initialize an empty mask to store the filtered segments
# filtered_segments = np.zeros_like(segmented_nuclei)

# # Initialize a counter for assigning new labels to filtered segments
# k = 1

# # Iterate through each connected region in the segmented nuclei
# for region in regionprops(segmented_nuclei):
#     if region.perimeter > 0:
#         # Compute roundness (circularity)
#         roundness = 4 * np.pi * region.area / (region.perimeter ** 2)

#         # Keep only regions that are sufficiently round
#         if roundness >= min_roundness:
#             filtered_segments[segmented_nuclei == region.label] = k
#             k += 1

# viewer_full.add_labels(filtered_segments, name="Filtered by Roundness")
# napari.run()
# im_out = filtered_segments.copy()

### Step 4: Watershed for Better Separation

In [ ]:
im_in=im_out
filtered_mask=im_in

# Compute the distance transform of the filtered mask
distance = distance_transform_edt(filtered_mask)

# Create markers for the watershed algorithm based on the distance transform
# Markers are created where the distance is greater than a small fraction of the maximum distance
markers = label(distance > 0.0001 * distance.max())

# Apply the watershed algorithm to segment nuclei
# The negative distance is used to ensure that the watershed grows from the markers
segmented_nuclei = watershed(-distance, markers, mask=filtered_mask)

viewer_full.add_labels(segmented_nuclei, name="Segmented Nuclei")
napari.run()
im_out=segmented_nuclei.copy()

### Output 
Creates a .tiff file with multiple pages. P1 is the original image, P2 is the ROI chosen as the tissue, P3 is the detected nuclei.

In [ ]:
filtered_segments=segmented_nuclei

In [ ]:
# Convert the original image array to an RGB image
rgb_im = Image.fromarray(imarray.astype(np.uint8), mode="RGB")

# Convert the ROI image array to an RGB image
rgb0_im = Image.fromarray(imarray0.astype(np.uint8), mode="RGB")

# Initialize an empty RGB array for the filtered segments
filtered_segments_rgb = np.zeros((np.shape(filtered_segments)[0], np.shape(filtered_segments)[1], 3))

# Generate random colors for each segment label
cmd = np.random.rand(np.max(filtered_segments) + 1, 3)
cmd[0, :] = [0.0, 0.0, 0.0]  # Ensure the background (label 0) is black

# Assign colors to each pixel based on its segment label
for i in range(1, np.shape(filtered_segments)[0]):
    for j in range(1, np.shape(filtered_segments)[1]):
        filtered_segments_rgb[i, j, 0] = int(cmd[filtered_segments[i, j], 0] * 255.0)  # Red channel
        filtered_segments_rgb[i, j, 1] = int(cmd[filtered_segments[i, j], 1] * 255.0)  # Green channel
        filtered_segments_rgb[i, j, 2] = int(cmd[filtered_segments[i, j], 2] * 255.0)  # Blue channel

# Convert the RGB array to uint8 format
filtered_segments_rgb = filtered_segments_rgb.astype('uint8')

# Convert the filtered segments RGB array to an image
b_im = Image.fromarray(filtered_segments_rgb, mode="RGB")

# Save the original image, ROI image, and filtered segments image in a single multi-page TIFF file
output_path = tiff_stem + "_output.tiff"
rgb_im.save(output_path, save_all=True, append_images=[rgb0_im, b_im])

# Print the output file path
print(f"TIFF file saved at: {output_path}")

### QUANTIFICATION

In [ ]:
#from scipy.ndimage import label, center_of_mass

# Get unique labels from the filtered segments (excluding the background label 0)
labels = np.unique(filtered_segments)
labels = labels[labels != 0]

# Print the total number of nuclei detected
print('TOTAL NUCLEI ' + str(len(labels+1)))

# Compute the centroids (barycenters) of each nucleus
barycenters = {l: center_of_mass(filtered_segments == l) for l in labels}

# Compute the area of each nucleus in square micrometers
areas = {l: np.sum(filtered_segments == l) * r_X * r_Y for l in labels}

# Convert barycenters to a dictionary of coordinates in micrometers
barycenter_coords = {k: (round(v[0], 2) * r_X, round(v[1], 2) * r_Y) for k, v in barycenters.items()}

# Calculate the total area of the image in square micrometers
fullA = np.prod(np.shape(mask_image)) * r_X * r_Y

# Calculate the total area of the ROI in square micrometers
roiA = np.sum(mask_image) * r_X * r_Y

# Print the total area of the image and the ROI
print("TOTAL AREA IMAGE %.2e um2" % fullA)
print("TOTAL AREA ROI %.2e um2" % roiA)

# Calculate the concentration of cells in the ROI (cells per square micrometer)
roiCON = (len(labels+1)) / roiA
roiCONmm=roiCON*1e6

# Print the cell concentration in the ROI
print("CELL CONCENTRATION in ROI %.2e cells/mm2" % roiCONmm)

In [ ]:
# Create a new Excel workbook
workbook = xlsxwriter.Workbook(tiff_stem + '.xlsx')

## FORMATS
# Define a format for headers (bold text with yellow background)
header = workbook.add_format({'bold': True})
header.set_bg_color('yellow')

# Define a format for floating-point numbers
floats = workbook.add_format({'num_format': '0.00'})

# Define a format for exponential numbers
exp = workbook.add_format()
exp.set_num_format(11)

## CELLS
# Create a worksheet for cell data
worksheet_cell = workbook.add_worksheet('Cells')

# HEADER
# Write the header row for the 'Cells' worksheet
worksheet_cell.write_row('A1:E1', ['#ID', 'X [um]', 'Y [um]', 'Area Nuclei [um2]'], header)

# CONTENT
# Write the data for each nucleus
for row, value in enumerate(labels):
    worksheet_cell.write(row + 1, 0, value)  # Write nucleus ID
    worksheet_cell.write(row + 1, 1, barycenter_coords[value][0], floats)  # Write X coordinate
    worksheet_cell.write(row + 1, 2, barycenter_coords[value][1], floats)  # Write Y coordinate
    worksheet_cell.write(row + 1, 3, areas[value], floats)  # Write area of the nucleus
    clear_output(wait=True)  # Clear the output to show progress
    print('NUCLEI ' + str(row + 1) + ' / ' + str(len(labels + 1)))  # Print progress

## ROI SHEET
# Create a worksheet for ROI data
worksheet_ROI = workbook.add_worksheet('ROI')

# HEADER
# Write the header row for the 'ROI' worksheet
worksheet_ROI.write_row('A1:E1', ['# NUCLEI', 'TOT AREA [um2]', 'ROI AREA [um2]', 'CONC NUCLEI in ROI [cells/mm2]'], header)

# CONTENT
# Write the ROI data
worksheet_ROI.write(1, 0, len(labels + 1))  # Write the total number of nuclei
worksheet_ROI.write(1, 1, fullA, exp)  # Write the total area of the image
worksheet_ROI.write(1, 2, roiA, exp)  # Write the total area of the ROI
worksheet_ROI.write(1, 3, roiCONmm, exp)  # Write the cell concentration in the ROI

# Close the workbook to save the file
workbook.close()